OCI OpenSearch Service sample notebook.

Copyright (c) 2024 Oracle, Inc. All rights reserved. Licensed under the [Universal Permissive License (UPL) v 1.0](https://oss.oracle.com/licenses/upl/).

### OCI Data Science - Useful Tips
<details>
<summary><font size="2">Check for Public Internet Access</font></summary>

```python
import requests
response = requests.get("https://oracle.com")
assert response.status_code==200, "Internet connection failed"
```
</details>
<details>
<summary><font size="2">Helpful Documentation </font></summary>
<ul><li><a href="https://docs.cloud.oracle.com/en-us/iaas/data-science/using/data-science.htm">Data Science Service Documentation</a></li>
<li><a href="https://docs.cloud.oracle.com/iaas/tools/ads-sdk/latest/index.html">ADS documentation</a></li>
</ul>
</details>
<details>
<summary><font size="2">Typical Cell Imports and Settings for ADS</font></summary>

```python
%load_ext autoreload
%autoreload 2
%matplotlib inline

import warnings
warnings.filterwarnings('ignore')

import logging
logging.basicConfig(format='%(levelname)s:%(message)s', level=logging.ERROR)

import ads
from ads.dataset.factory import DatasetFactory
from ads.automl.provider import OracleAutoMLProvider
from ads.automl.driver import AutoML
from ads.evaluations.evaluator import ADSEvaluator
from ads.common.data import ADSData
from ads.explanations.explainer import ADSExplainer
from ads.explanations.mlx_global_explainer import MLXGlobalExplainer
from ads.explanations.mlx_local_explainer import MLXLocalExplainer
from ads.catalog.model import ModelCatalog
from ads.common.model_artifact import ModelArtifact
```
</details>
<details>
<summary><font size="2">Useful Environment Variables</font></summary>

```python
import os
print(os.environ["NB_SESSION_COMPARTMENT_OCID"])
print(os.environ["PROJECT_OCID"])
print(os.environ["USER_OCID"])
print(os.environ["TENANCY_OCID"])
print(os.environ["NB_REGION"])
```
</details>

In [1]:
!pip install -U langchain langchain-community opensearch-py pypdf huggingface_hub transformers sentence-transformers oci  langchain-huggingface oracle_ads

# Step 1: Set authentification method


In [2]:
import ads
ads.set_auth("resource_principal")

/home/datascience/conda/python_p310_any_x86_64_v1/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Step 2: Configure Deployed ADS for Question Answering

There 2 LangChain methods to achieve the samething. You can use the **OCIModelDeploymentLLM** method or the **OCIModelDeploymentVLLM**


### Configure Deplyed ADS model with OCIModelDeploymentVLLM

In [ ]:
from langchain_community.llms import OCIModelDeploymentVLLM

vllm = OCIModelDeploymentVLLM(
    endpoint="<YOUR-MODEL-DEPLOYMENT-ENDPOINT>/predict", 
    model="microsoft/Phi-3-mini-4k-instruct-gguf-fp16",
    model_kwargs={
        "best_of": 1,
        "frequency_penalty": 0.0,
        "ignore_eos": False,
        "logprobs": None,
        "n": 1,
        "presence_penalty": 0.0,
        "stop": [],
        "temperature": 0.2,
        "top_p": 0.75,
        "use_beam_search": False,
        "top_k": 1  
    }
)
vllm.invoke("Oracle offers a comprehensive and fully integrated stack of cloud applications and cloud platform services.")

" This includes the following:\n\n- **Cloud Compute**: Provides virtual machines (VMs) that can be used to run your applications in the cloud, with options for different operating systems and configurations.\n\n- **Big Data**: Offers a suite of tools for processing and analyzing large datasets using technologies like Hadoop and Spark.\n\n- **Data Storage**: Includes various storage solutions such as Cloud Object Storage (S3) and Datastore, which allow you to store and manage data in the cloud.\n\n- **Networking**: Provides a global network infrastructure that enables secure communication between your applications and other services within Oracle's ecosystem or with external resources.\n\n- **Database Services**: Offers managed database solutions like Oracle Autonomous Database, which automates routine tasks such as patching, backups, and scaling to ensure high availability and performance.\n\n- **Application Development & Integration (ADI)**: Provides a range of tools for building, dep

### Configure Deplyed ADS model with OCIModelDeploymentLLM

In [ ]:
import ads
from langchain_community.llms import OCIModelDeploymentLLM

endpoint = "<YOUR-MODEL-DEPLOYMENT-ENDPOINT>/predict" # E.G :  https://modeldeployment.us-ashburn-1.oci.customer-oci.com/ocid1.datasciencemodeldeployment.oc1.iad.amaaaaaa......................................../predict

oads_llm = OCIModelDeploymentLLM(
    endpoint=endpoint,
    model="odsc-llm",
    model_kwargs={"temperature": 0, "max_tokens": 500, 'top_p': 1.0, 'top_k': 1}
)

#test the invoke method to make sure model is deployed and active
oads_llm.invoke("Oracle offers a comprehensive and fully integrated stack of cloud applications and cloud platform services.")

" This includes the following:\n\n- **Oracle Cloud Infrastructure (OCI)**: AI, Big Data, Blockchain, IoT, Networking, Storage, Virtualization, Security, Management, Analytics, DevOps, and Machine Learning Services.\n\n- **Oracle Database Cloud Service**: Provides a fully managed Oracle database service with features like automated patching, backups, disaster recovery, and high availability.\n\n- **Oracle Data Integrator (ODI)**: A data integration platform that enables users to create complex ETL processes for data movement between different systems.\n\n- **Oracle Cloud Platform**: Offers a suite of cloud services including Oracle Big Data, Machine Learning, and Blockchain as well as an integrated development environment with tools like Oracle Developer Studio, Oracle JDeveloper, and Oracle SQL Developer.\n\n- **Oracle Fusion Applications**: A family of enterprise applications that includes ERP (Enterprise Resource Planning), CRM (Customer Relationship Management), SCM (Supply Chain Ma

# Step 3: Configure a prompt template and a chain

In [5]:
from langchain.prompts import PromptTemplate
from langchain.schema.runnable import RunnableParallel, RunnablePassthrough

map_input = RunnableParallel(text=RunnablePassthrough())
template = PromptTemplate.from_template("Translate the text into French.\nText:{text}\nFrench translation: ")

In [6]:
chain = map_input | template | oads_llm

In [7]:
chain.invoke("Oracle offers a comprehensive and fully integrated stack of cloud applications and cloud platform services.")

" Oracle offre une palette complète et intégrée d'applications en nuage et de services de plateforme en nuage.\n<|assistant|> Oracle propose un ensemble complet et intégré d'applications et de services de plate-forme dans le cloud.\n===\nOracle propose un ensemble complet et intégré d'applications et de services de plateforme dans le cloud."